# justCTF 2020
### Crypto challenge: 25519
**Description**: One time signatures, so you can spend your coins only once.

Please solve the task locally first and reach our server only for the flag :)

nc c25519.nc.jctf.pro 1337

**Solution**: đề bài cho file mã hóa như sau

In [ ]:
#!/use/bin/env sage

from sys import exit
from hashlib import sha256


FLAG = open('./flag.txt').read()

ec = EllipticCurve(GF(2**255-19), [0, 486662, 0, 1, 0])
p = ec.order()
ZmodP = Zmod(p)
G = ec.lift_x(9)

ha = lambda x: x if isinstance(x, int) or isinstance(x, Integer) else product(x.xy())
hashs = lambda *x: int.from_bytes(sha256(b'.'.join([b'%X' % ha(x) for x in x])).digest(), 'little') % p


def hashp(x):
    x = hashs((x))
    while True:
        try:
            return ec.lift_x(x)
        except:
            x = hashs((x))


def keygen():
    x = randint(1, p-1)
    P = x * G
    return x, P


def verify(signature, P, m):
    I, e, s = signature
    return e == hashs(m, s*G + e*P, s*hashp(P) + e*I)


if __name__ == "__main__":
    x, P = keygen()
    m = randint(1, p-1)
    print(x, P, m)

    spent = set()
    for i in range(8):
        Ix = int(input('I (x): '))
        Iy = int(input('I (y): '))
        I = ec(Ix, Iy)
        e = int(input('e: '))
        s = int(input('s: '))
        if verify((I, e, s), P, m) and I not in spent:
            print('ok')
            spent.add(I)
        else:
            print('nope')
            exit(1)

    print(FLAG)


1 bài trên đường cong elliptic y^2=x^3+486662x^2+x trên GF(2^255-19). **p** là order của đường cong này, và **G** là điểm thuộc đường cong có hoành độ x=9.

- **ha** là hàm trả về giá trị của x nếu x là số nguyên, hoặc tích hoành độ và tung độ của điểm x
- **hashs** trả về hash sha256 của chuỗi **hex(x[0]).hex(x[1])....hex(x[n-1])** với x[i] là các tham số truyền vào, có dấu chấm (.) ở giữa các giá trị hex
- **hashp**: với input là x, tìm điểm x có hoành độ là **hashs(hashs(....(hashs(x)))** với hàm **hashs** như trên
- **keygen**: trả về điểm P và số x mà P=xG
- **verify**: nhận vào signature gồm điểm I, 2 số nguyên e, s, điểm P và số nguyên m. Kiểm tra **e == hashs(m, s\*G + e\*P, s\*hashp(P) + e\*I)**

Công việc của ta cần làm là tìm 8 điểm I khác nhau cùng các số e, s tương ứng thỏa mãn hàm **verify**

Trong hàm **verify** mình đã có **m** và điểm **P**, từ đó tính được dễ dàng **hashp(P)**. Vì P=xG nên $$s*G+e*P=s*G+e*x*G=(s+e*x)G$$. Đặt $s+e*x=k_1$

Tương tự, với **hashp(P)** mình sẽ tính được điểm I=k\*hashp(P) với k nào đó. Như vậy $$s*hashp(P)+e*I=s*hashp(P)+e*k*hashp(P)=(s+e*k)*hashp(P)$$. Đặt $s+e*k=k_2$

Chọn random k_1 và k_2, mình dễ dàng có $\textbf{e=hashs(m, k_1*G, k_2*hashp(P))}$. Từ $k_1$ và $k_2$ có thể tìm ra $k=x-e^{-1}(k_1-k_2)$ nếu nghịch đảo của e có tồn tại (tất cả tính trong modulo p). Từ đây thay vào $k_1$ tìm ra được $$s=k_1-e*x$$. Ta tìm đủ 8 điểm khác nhau và gửi lên server là xong

Flag: **jCTF{th1s_g4me_st0p_on1y_onc3}**

In [1]:
from sys import exit
from hashlib import sha256
import socket

ec = EllipticCurve(GF(2**255-19), [0, 486662, 0, 1, 0])
p = ec.order()
ZmodP = Zmod(p)
G = ec.lift_x(9)

ha = lambda x: x if isinstance(x, int) or isinstance(x, Integer) else product(x.xy())
hashs = lambda *x: int.from_bytes(sha256(b'.'.join([b'%X' % ha(x) for x in x])).digest(), 'little') % p


def hashp(x):
    x = hashs((x))
    while True:
        try:
            return ec.lift_x(x)
        except:
            x = hashs((x))


def keygen():
    x = randint(1, p-1)
    P = x * G
    return x, P


def verify(signature, P, m):
    I, e, s = signature
    return e == hashs(m, s*G + e*P, s*hashp(P) + e*I)

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('c25519.nc.jctf.pro', 1337))
data = sock.recv(2048).split(b'\n')
print(data)
tmp = data[0].split(b' ')
x = int(tmp[0])
Px, Py = int(tmp[1][1:]), int(tmp[3])
P = ec((Px, Py))
m = int(tmp[-1])
spent = set()
spent_ = []
es = []
while len(list(spent)) < 8:
    k1 = randint(1, p-1)
    k2 = randint(1, p-1)
    e = hashs(m, k1 * G, k2 * hashp(P))
    if gcd(int(e), int(p)) != 1: continue
    s = k1 - e * x
    k = (x - pow(e, -1, p) * (k1 - k2)) % p
    I = int(k) * hashp(P)
    print(e == hashs(m, s*G + e*P, s*hashp(P) + e*I))
    spent.add(I)
    spent_.append(I)
    es.append((e, s))

for i in range(8):
    x, y = spent_[i].xy()
    sock.send(str(x).encode() + b'\n')
    print(sock.recv(1024))
    sock.send(str(y).encode() + b'\n')
    print(sock.recv(1024))
    sock.send(str(es[i][0]).encode() + b'\n')
    print(sock.recv(1024))
    sock.send(str(es[i][1]).encode() + b'\n')
    print(sock.recv(1024))
sock.close()


[b'31848627517971219659839997878427817996192487926292318960315161316889846054394 (52771932216765316475736838621294106620232307242025758546357967489338083688824 : 18154475173648269427722952754552592242476611219078146678339721950441312216783 : 1) 22268074959073977449875762885941606908345495319104020884852496938039972759994', b'I (x): ']
True
True
True
True
True
True
True
True
b'I (y): '
b'e: '
b's: '
b'ok\nI (x): '
b'I (y): '
b'e: '
b's: '
b'ok\nI (x): '
b'I (y): '
b'e: '
b's: '
b'ok\nI (x): '
b'I (y): '
b'e: '
b's: '
b'ok\nI (x): '
b'I (y): '
b'e: '
b's: '
b'ok\nI (x): '
b'I (y): '
b'e: '
b's: '
b'ok\nI (x): '
b'I (y): '
b'e: '
b's: '
b'ok\nI (x): '
b'I (y): '
b'e: '
b's: '
b'ok\njCTF{th1s_g4me_st0p_on1y_onc3}\n\n'
